In [1]:
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 1

In [19]:
from django.core.files.storage import default_storage
from django.core.files.storage import get_storage_class

from homepage.blogs.models import BlogImage
from homepage.blogs.utils import storage_walk_paths

In [9]:
def get_models_paths():
    paths = set()
    for image in BlogImage.objects.all():
        for path in image.get_all_paths():
            paths.add(path)
    for video in BlogVideo.objects.all():
        for path in video.get_all_paths():
            paths.add(path)
    for misc_file in BlogFile.objects.all():
        for path in misc_file.get_all_paths():
            paths.add(path)
    return paths

In [10]:
%%time
paths_from_models = get_models_paths()

In [12]:
len(paths_from_models)

4143

In [7]:
def get_paths(storage):
    paths = {}
    for path in storage_walk_paths(storage):
        size = storage.size(path)
        paths[path] = size
    return paths

In [8]:
%%time
s3 = get_storage_class('storages.backends.s3boto3.S3Boto3Storage')()
s3_paths = get_paths(s3)

In [13]:
len(s3_paths)

7527

In [15]:
stale_s3 = {}
for path, size in s3_paths.items():
    if path not in paths_from_models:
        print(path)
        stale_s3[path] = size
print('stale s3 size: {} Mb'.format(sum(stale_s3.values()) / 2 ** 20))

CACHE/images/blogs_videos/poster/poster__bhlb1sq/fc015c0bfcdcfedf65a15b7c34130197.jpg
CACHE/images/blogs_videos/poster/poster_3vyywq7q/70e1671f28a2331880f50ca64cfbcece.jpg
CACHE/images/blogs_videos/poster/poster_ejhpfiqo/176c2380a7ea1176c0a3b226cc79491e.jpg
CACHE/images/blogs_videos/poster/poster_ixij00hf/96bbe706c00316e0dd4db10304229d6f.jpg
CACHE/images/blogs_videos/poster/poster_esrjrq90/3e9eb93d8af10da73d80ffb3091fe789.jpg
CACHE/images/blogs_videos/poster/poster_kccd027q/0415c2585a4b94295c9118f831bd250c.jpg
CACHE/images/blogs_videos/poster/poster_l_v7yxnn/6ccb2e4d1438c70c56c2a0bd7c0c8c16.jpg
CACHE/images/blogs_videos/poster/poster_hr3yjsza/4d15296fc1dc2c25e9d7fd9769878125.jpg
CACHE/images/blogs_videos/poster/Bildschirmfoto_2017-11-01_um_14.55.14/1234f029c4bdd6402996efa1111b0ffe.jpg
CACHE/images/blogs_videos/poster/poster_09pvlp9x/eeda68482da32e0cf11874b168290870.jpg
CACHE/images/blogs_videos/poster/poster_mxvi3o33/17595d8da32e95d01b802d67ea1e0893.jpg
CACHE/images/blogs_videos/poster

CACHE/images/blogs_images/originals/schloss-dyck-2018-18/e8471d825fa5b73f8488830dd0d1eaa9.jpg
CACHE/images/blogs_images/originals/DSC_0435/4fd33746ff12fbcd44ee2582d515911a.jpg
CACHE/images/blogs_images/originals/DSC_0435/854f5561cc7f4f840b8e91e71ecd37af.jpg
CACHE/images/blogs_images/originals/DSC_0435/949d9aed1e82fe48722c60fa7328fd0c.jpg
CACHE/images/blogs_images/originals/DSC_0435/99ec1af812563c7b18429cad3f1d0685.jpg
CACHE/images/blogs_images/originals/DSC_0435/d04a5bdaac9764bf32bae123c64fd2d1.jpg
CACHE/images/blogs_images/originals/DSC_0435/f64e948360962490da2bea6a23029b3f.jpg
CACHE/images/blogs_images/originals/fullsizeoutput_3a2e/1e675cbf0da0e5b74ea8051d2b4ff4d9.jpeg
CACHE/images/blogs_images/originals/fullsizeoutput_3a2e/8dab750cbfa9a9b0b19b403df252d72b.jpeg
CACHE/images/blogs_images/originals/fullsizeoutput_3a2e/a801b723bdb31014825733856cda79b7.jpeg
CACHE/images/blogs_images/originals/fullsizeoutput_3a2e/ccb320c6e2ce7e76e4c92b38c0096265.jpeg
CACHE/images/blogs_images/originals/fu

blogs_videos/poster/poster_esrjrq90.jpg
blogs_videos/poster/poster_ixij00hf.jpg
blogs_videos/poster/poster_iyd83om1.jpg
blogs_videos/poster/poster_kfs46ydl.jpg
blogs_videos/poster/poster_m77t2085.jpg
blogs_videos/poster/poster_mxvi3o33.jpg
blogs_videos/poster/poster_n7u_xvbq.jpg
blogs_videos/poster/poster_owugj5i9.jpg
blogs_videos/poster/poster_qdk02rgb.jpg
blogs_videos/poster/poster_wime8po2.jpg
blogs_videos/6Ob6vhJfQSud1PBhVlqeQ.MOV
blogs_videos/Bildschirmfoto_2018-02-02_um_18.02.23.png
blogs_videos/Q0nsfud4RAu8cVuhhwIucw.MOV
blogs_videos/T1ZFccXTJmFSda1E8i1VA.MOV
blogs_videos/claas_zieht_sich_am_sofa_hoch.mp4
blogs_videos/fruehstueckstimelapse.mp4
blogs_videos/mQoKPNdRYWUH0rb11qvnw.MOV
blogs_videos/schnullern_will_gelernt_sein_compressed.mp4
blogs_videos/uC9kBTpgTZCuYBmm4LkvDw.MOV
blogs_images/originals/Pool-Theine-2018-1.jpg
blogs_images/originals/Pool-Theine-2018-10.jpg
blogs_images/originals/Pool-Theine-2018-11.jpg
blogs_images/originals/Pool-Theine-2018-12.jpg
blogs_images/origi

In [17]:
def get_paths(storage):
    paths = {}
    for path in storage_walk_paths(storage):
        size = storage.size(path)
        paths[path] = size
        # print(path, size / 2 ** 20)
    return paths

In [21]:
print('stale locale')
locale_paths = get_paths(default_storage)
stale_locale = {}
for path, size in locale_paths.items():
    if path not in paths_from_models:
        print(path)
        stale_locale[path] = size
print('stale locale size: {} Mb'.format(sum(stale_locale.values()) / 2 ** 20))
print(len(stale_locale))

stale locale
CACHE/images/cast_images/originals/iqHs41HGQsW2Ma1jN9iCag/d3a6eafeee44c4d97aec5bce335ddcad.jpg
CACHE/images/cast_images/originals/iqHs41HGQsW2Ma1jN9iCag/0668b8a10bc853cc5fd08f89f5dc5e8d.jpg
CACHE/images/cast_images/originals/iqHs41HGQsW2Ma1jN9iCag/9d22f735ab646ca92c2534a1ed13c1d2.jpg
CACHE/images/cast_images/originals/iqHs41HGQsW2Ma1jN9iCag/e91b3802f00a3607cef2c51899260ff2.jpg
CACHE/images/cast_images/originals/iqHs41HGQsW2Ma1jN9iCag/fc6f3defc1f10dbe04723ad6680e0584.jpg
CACHE/images/cast_images/originals/iqHs41HGQsW2Ma1jN9iCag/18310342dfb424495649d7d1298cc74c.jpg
CACHE/images/blogs_images/originals/fullsizeoutput_3ac6/439f6f16ab42c2baa86efdd32422f9a3.jpeg
CACHE/images/blogs_images/originals/fullsizeoutput_3ac6/7a47fda216aed1580fcc3b29e36c0d5a.jpeg
CACHE/images/blogs_images/originals/fullsizeoutput_3ac6/c1ceec21a108dfad28f93802ee5fa41a.jpeg
CACHE/images/blogs_images/originals/fullsizeoutput_3ac6/30f3bd98d8b73f9e30f4f7cf6b98b232.jpeg
CACHE/images/blogs_images/originals/fulls

In [22]:
image = BlogImage.objects.all().order_by("-created")[0]

In [23]:
image.original

<ImageFieldFile: blogs_images/originals/fullsizeoutput_463c.jpeg>

In [30]:
list(image.get_all_paths())[0]

'CACHE/images/blogs_images/originals/fullsizeoutput_463c/e28d168c88ee9a0be49d6a6b26f3bde5.jpeg'

In [26]:
list(locale_paths)[0]

'CACHE/images/cast_images/originals/iqHs41HGQsW2Ma1jN9iCag/d3a6eafeee44c4d97aec5bce335ddcad.jpg'

In [31]:
bp = BlogPost.objects.get(slug="Picknick-vor-dem-Hochofen-5")

In [33]:
image = bp.images.all()[0]

In [35]:
list(image.get_all_paths())[0] in s3_paths

True